[Kinetica](https://www.kinetica.com/) is a database with integrated support for vector similarity search.

It supports:

- exact and approximate nearest neighbor search
- L2 distance, inner product, and cosine distance

This notebook shows how to use the Kinetica vector store (`Kinetica`).

This needs an instance of Kinetica which can easily be setup using the instructions given here - [installation instruction](https://www.kinetica.com/developer-edition/).

In [ ]:
pip install -qU langchain-kinetica

We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [1]:
import getpass
import os

from langchain_openai import OpenAIEmbeddings

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

You must set the database connection in the following environment variables. If you are using a virtual environment you can set them in the `.env` file of the project:

* `KINETICA_URL`: Database connection URL (e.g. `http://localhost:9191`)
* `KINETICA_USER`: Database user
* `KINETICA_PASSWD`: Secure password.

In [5]:
# Kinetica needs the connection to the database.
# Set these environment variables:
from gpudb import GPUdb

kdbc = GPUdb.get_connection(enable_ssl_cert_verification=True)

2026-02-26 12:58:16.802 INFO     [GPUdb] Connected to Kinetica! (host=https://demo72.kinetica.com/_gpudb api=7.2.3.6 server=7.2.3.8)


In [6]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for"
    " breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast"
    ", with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful"
    ", agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to"
    " fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]
uuids

['be80a0ec-c5cf-46f1-981a-143a03c7a0be',
 '83498271-68f5-44a1-a3ae-b4c9dd3cb6bb',
 '2c4a70a0-6cc9-41d9-b1c2-da9d281a20cf',
 '789ebfa4-fe99-49d0-85a9-c540edd4758c',
 '81cde068-d9e6-4c8e-b5ad-fd5ecafe359b',
 '5cc0b804-2f97-42b1-8f4f-c5e39df33c02',
 '85b9aade-6830-48f9-9198-2282ec49fd19',
 '5e6900bc-2d4c-4e95-bd06-3fed8b53960b',
 'ff5296ce-23d7-45b6-a824-ae60bde99f5f',
 '328e03cc-be7f-45e3-811f-be996c657b28']

## Similarity search with euclidean distance (Default)

The Kinetica Module will try to create a table with the name of the collection.
So, make sure that the collection name is unique and the user has the permission to create a table.

In [9]:

from langchain_kinetica import KineticaVectorstore

COLLECTION_NAME = "langchain_example"
SCHEMA_NAME = "test"

vectorstore = KineticaVectorstore(
    kdbc=kdbc,
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
    schema_name=SCHEMA_NAME,
)

vectorstore.add_documents(documents=documents, ids=uuids)

print()
print("Similarity Search")
results = vectorstore.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

print()
print("Similarity search with score")
results = vectorstore.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, emb_filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")


Similarity Search
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]

Similarity search with score
* [SIM=0.945397] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


## Working with vectorstore

### Adding documents

Above, we created a vectorstore from scratch. However, often times we want to work with an existing vectorstore.
In order to do that, we can initialize it directly.

In [10]:
vectorstore = KineticaVectorstore(
    kdbc=kdbc,
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
    schema_name=SCHEMA_NAME,
)

# We can add documents to the existing vectorstore.
vectorstore.add_documents([Document(page_content="foo")])

docs_with_score = vectorstore.similarity_search_with_score("foo")

print(f"First result: {docs_with_score[0]}")
print(f"Second result: {docs_with_score[1]}")

First result: (Document(metadata={}, page_content='foo'), 0.0)
Second result: (Document(metadata={}, page_content='foo'), 0.001588501618243754)


### Overriding a vectorstore

If you have an existing collection, you override it by doing `from_documents` and setting `pre_delete_collection` = True

In [11]:
vectorstore = KineticaVectorstore.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    schema_name=SCHEMA_NAME,
    kdbc=kdbc
)

docs_with_score = vectorstore.similarity_search_with_score("foo")
docs_with_score[0]

(Document(metadata={}, page_content='foo'), 0.001483637373894453)

### Using a VectorStore as a retriever

In [12]:
from langchain_core.vectorstores.base import VectorStoreRetriever

retriever: VectorStoreRetriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['KineticaVectorstore', 'OpenAIEmbeddings'], vectorstore=<langchain_kinetica.vectorstores.KineticaVectorstore object at 0x10cee6570>, search_kwargs={})